In [8]:
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse, parse_qs
import google.generativeai as genai

In [9]:
def get_all_transcripts(video_id):
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
        all_transcripts = []
        for transcript in transcript_list:
            all_transcripts.append(transcript.language_code)
        return all_transcripts
    except Exception as e:
        return f"Error fetching transcripts: {e}"

In [10]:
def get_youtube_transcript(youtube_url):
    video_id = parse_qs(urlparse(youtube_url).query).get("v")
    if not video_id:
        raise ValueError("Invalid YouTube URL. Couldn't find video ID.")
    video_id = video_id[0]
    try:
        transcript_data = YouTubeTranscriptApi.get_transcript(video_id, languages=get_all_transcripts(video_id))
        transcript = " ".join([t["text"] for t in transcript_data])
        return transcript
    except Exception as e:
        return f"Error fetching transcript: {e}"

In [ ]:
# Replace with a real YouTube URL.
yt_url = input("Enter the Youtube video Link:")

# Fetch the transcript BEFORE using it in other functions
transcript = get_youtube_transcript(yt_url)

# Safely access Gemini API Key
GOOGLE_API_KEY = input('Enter your Google API Key')

genai.configure(api_key=GOOGLE_API_KEY)

# Initialize the Gemini Pro model
model = genai.GenerativeModel('models/gemini-2.0-flash')

In [12]:
def generate_prompt(transcript, question=None, video_type="general", is_hindi=False, task="chat", chat_history=None):
    """
    Generates a prompt for Gemini model based on transcript type and task for multi-turn chat.

    Parameters:
    - transcript: str, the video transcript
    - question: str, the user's question
    - video_type: str, e.g., 'educational', 'motivational', etc.
    - is_hindi: bool, whether the transcript is in Hindi
    - task: str, should be 'chat'
    - chat_history: list of tuples (user_message, bot_response)

    Returns:
    - str: Formatted prompt for Gemini
    """
    base_prompts = {
        "educational": "You are a helpful tutor. Engage in a multi-turn interactive Q&A session based on the transcript.",
        "motivational": "You are a motivational content explainer. Engage in a multi-turn conversation based on the transcript.",
        "product": "You are a product assistant helping users understand tech reviews or tutorials. Engage in a multi-turn conversation based on the transcript.",
        "news": "You are a factual news analyst. Engage in a multi-turn conversation based on the transcript.",
        "general": "You are a smart assistant that explains things clearly. Engage in a multi-turn conversation based on the transcript."
    }

    role = base_prompts.get(video_type.lower(), base_prompts["general"])

    prompt = f"""{role}
Based on the transcript below, answer the user's questions and engage in a conversation.

Transcript:
{transcript[:12000]}

"""

    if chat_history:
        for user_msg, bot_msg in chat_history:
            prompt += f"User: {user_msg}\nBot: {bot_msg}\n"

    prompt += f"User: {question}\nBot: "

    return prompt

In [ ]:
def chat_with_transcript(model, transcript, video_type="general"):
    """
    Initiates a multi-turn chat with the Gemini model based on a video transcript.

    Parameters:
    - model: Gemini model object
    - transcript: full transcript from video
    - video_type: genre of video e.g., "motivational", "educational", "news", etc.

    Returns:
    - None (prints the chat interaction)
    """
    chat_history = []
    while True:
        question = input("User: ")
        if question.lower() in ["exit", "quit", "bye"]:
            print("Bot: Goodbye!")
            break

        prompt = generate_prompt(transcript, question, video_type, chat_history=chat_history)
        response = model.generate_content(prompt)
        answer = response.text
        # print(f"User:{question}\n")
        print(f"Bot: {answer}")
        chat_history.append((question, answer))

In [14]:
# Usage
video_type = "educational"  # Change to "motivational", "news", "product", or "general"
chat_with_transcript(model, transcript, video_type)

User: give the context of the video
Bot: Okay! Based on the transcript, here's the context I can gather about the video:

*   **It's a song:** The transcript is clearly lyrics to a song, likely in Marathi (based on some of the words).
*   **Theme:** The song seems to be about partying, dancing, and enjoying music played by a DJ. It mentions drinking ("quarter," "glass"), dancing ("disco dancing," "lezim"), and a lively atmosphere.
*   **DJ Focus:** The lyrics repeatedly call for the "Dolby वाल्या (Wala)" DJ, emphasizing the importance of the DJ and the sound system (Dolby).
*   **Cultural Elements:** The song includes cultural references like "Varati" (wedding procession), "Dandia," "Govinda" (likely referring to the Dahi Handi festival), and "Pola" (a festival).
*   **Energy:** The language is energetic and informal, suggesting a fun and festive occasion.

So, in summary, the video is likely a recording of a high-energy Marathi party song that celebrates music, dancing, and the role o